<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - Popular MRR</h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender
from lenskit.algorithms.basic import Popular
import pandas as pd
import numpy as np

ratings = pd.read_csv('D:\\Escritorio\\UM\\Tesis\\ML 1M\\ratings.dat', sep='::',engine='python',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo)
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) 
    users = test.user.unique() 
    recs = batch.recommend(fittable, users, 100)
    recs['Algorithm'] = aname
    return recs

In [2]:
def model_trainer():
    all_recs = []
    test_data = []
    pop = Popular(selector=None) # define algorithm

    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test) # save testing data
        all_recs.append(eval('Popular', pop, train, test))

    all_recs = pd.concat(all_recs, ignore_index=True)
    test_data = pd.concat(test_data, ignore_index=True)

    rla = topn.RecListAnalysis()
    rla.add_metric(topn.recip_rank)
    mrr = rla.compute(all_recs, test_data)
    return mrr['recip_rank'].mean()

In [4]:
%%time
array = []
for i in range(1,100,1):
    array.append(model_trainer())

Wall time: 46min 12s


In [5]:
metric = pd.DataFrame(array)
metric.columns = ['MRR']
metric.head(5)

,MRR
0,0.392401
1,0.396792
2,0.396348
3,0.396200
4,0.393749


In [6]:
metric.sort_values(by=['MRR'],ascending=False).head(10)

,MRR
73,0.402202
27,0.400647
67,0.400278
34,0.400075
91,0.399390
22,0.399162
28,0.398564
82,0.398502
41,0.398440
23,0.398353


In [7]:
metric.to_csv('metric_MRR.csv', index=False)

In [10]:
all_recs.head(10)

,item,score,user,rank,Algorithm
0,50,560.0,19,1,Popular
1,100,490.0,19,2,Popular
2,181,485.0,19,3,Popular
3,294,466.0,19,4,Popular
4,286,461.0,19,5,Popular
5,1,427.0,19,6,Popular
6,300,415.0,19,7,Popular
7,121,413.0,19,8,Popular
8,174,402.0,19,9,Popular
9,127,395.0,19,10,Popular


In [11]:
all_recs.item.unique()

array([  50,  100,  181,  294,  286,    1,  300,  121,  174,  127,    7,
         98,   56,  237,  117,  222,  172,  204,  405,   79,  151,  173,
         69,  748,  269,  168,    9,  257,  318,   25,  195,  423,   15,
         22,  276,   96,  328,  302,  216,  183,  118,   89,  234,   64,
        176,   28,  191,  275,  111,   12,  742,  357,  135,   82,   97,
        546,  289,  333,  238,  186,   70,  475,  196,  132,  323,  125,
        483,  228,  194,  185,  144,   11,  197,  245,  301,  496,  282,
        273,  568,   71,  182,  471,  265,  161,  179,  427,  235,  322,
        143,  678,  271,  180,  435,   95,   88,  175,  508,  215,  603,
        187,  313,  210,  202,  268,    4,  211,  385,  515,  597,  230,
        403,  272,  208,  250,  588,  527,  134,  474,  298,  209,  393,
        124,  274,  340,  284,   14,  751,  147,  288,  153,  655,    8,
        200,  258,  480,  307,   13,  203,  591,  327,   23,  514,  479,
        845,  566,  732,  326,  419,   58,   99,  5

In [12]:
 ratings.groupby("item").count()["rating"].sort_values(ascending=False).head(20)

item
50     583
258    509
100    508
181    507
294    485
286    481
288    478
1      452
300    431
121    429
174    420
127    413
56     394
7      392
98     390
237    384
117    378
172    367
222    365
313    350
Name: rating, dtype: int64